In [16]:
import requests
from io import BytesIO
from pypdf import PdfReader
from google import genai
from fpdf import FPDF
import unicodedata

In [17]:
def limpiar_texto(texto):
    return unicodedata.normalize("NFKD", texto).encode("latin-1", "ignore").decode("latin-1")

In [2]:
url_gaceta="https://www.congresocdmx.gob.mx/media/documentos/ad0ce85ba0168324a6c67e1ef45544ea237072d2.pdf"

In [20]:
response= requests.get(url_gaceta)
file_like= BytesIO(response.content)
reader = PdfReader(file_like)

start_page = 4
end_page = 229
texto_ppg = ""

for page in range(start_page - 1, min(end_page, len(reader.pages))):
    texto_ppg += reader.pages[page].extract_text() + "\n"

In [21]:
texto_ppg = texto_ppg.replace("\n", " ")
texto_ppg

'2 Programa Provicional de Gobierno del Gobierno de la Ciudad de México Este programa provisional tiene vigencia de un  año, a la espera de la entrada en vigor del Plan  General de Desarrollo de la Ciudad de México  que será la base para la creación del próximo  Programa General de Gobierno del Gobierno de   la Ciudad de México que tendrá vigor hasta 2030. Construyamos juntas y juntos,  la Capital de la Transformación  3 Programa Provicional de Gobierno del Gobierno de la Ciudad de México Índice Introducción           3  Principios           14   Programa Provisional de Gobierno del Gobierno de la Ciudad de México   15 Ciudad con Buen Gobierno y Cercano a la Gente     16 Ciudad con Seguridad y Paz        21 Ciudad con Derecho al Agua y al Saneamiento      36 Ciudad con Ordenamiento Territorial y Visión Metropolitana    45 Ciudad con Suelo y Vivienda Accesible y Adecuada     52 Ciudad con Movilidad Integrada y Sustentable      59 Ciudad Sustentable, preparada frente al Cambio Climático 

In [48]:
#Salvar texto en un txt
with open("ppg_text.txt", "w", encoding="utf-8") as f:
    f.write(limpiar_texto(texto_ppg))

In [22]:
#Leer credenciales de google

with open('C:/cred_google_gemini.txt', 'r') as file:
    llave = file.read()

In [23]:
client=genai.Client(api_key=llave)

In [63]:
# Generar el contenido con el modelo
response = client.models.generate_content( 
    model="gemini-2.0-flash",
    contents="""Extrae los objetivos y estrategias de cada eje temático. No omitas ningún eje. No me des ninguna introducción diciéndome que "aquí tengo la información", solo lo que te pido.

Ciudad con Buen Gobierno y Cercano a la Gente

Ciudad con Seguridad y Paz

Ciudad con Derecho al Agua y al Saneamiento

Ciudad con Ordenamiento Territorial y Visión Metropolitana

Ciudad con Suelo y Vivienda Accesible y Adecuada

Ciudad con Movilidad Integrada y Sustentable

Ciudad Sustentable, preparada frente al Cambio Climático y con Bienestar

Ciudad Resiliente, Previsora y Protectora ante los Riesgos y Desastres

Ciudad Pluricultural y Justa para los Pueblos y Barrios Originarios y Comunidades Indígenas Residentes

Ciudad con Economía a Favor de la Gente

Ciudad Próspera con Empleo

Ciudad con Turismo Sostenible y Responsable, Equitativo y Accesible

Ciudad con Finanzas Sanas y Alta Calidad en el Ejercicio del Gasto

Ciudad Cuidadora y con Igualdad Social

Ciudad que Produce Salud

Ciudad con Deporte y Actividad Física




El documento es el siguiente:
""" + texto_ppg
)


resumen_ppg_1=response.text

In [64]:
response = client.models.generate_content( 
    model="gemini-2.0-flash",
    contents="""Extrae los objetivos y estrategias de cada eje temático. No omitas ningún eje. No me des ninguna introducción diciéndome que "aquí tengo la información", solo lo que te pido.
    
Ciudad de UTOPÍAS
    
Ciudad con Educación, Ciencia y Tecnología

Ciudad Capital Cultural

Ciudad Feminista y con Igualdad Sustantiva

Ciudad de las Niñas y los Niños Comprometida con el Interés Superior de la Infancia

Ciudad de Juventudes Libres y Diversas

Ciudad Solidaria e Intercultural, Garante de los Derechos de las Poblaciones en Movilidad Humana

Ciudad Incluyente de la Diversidad Sexual y de Género

Ciudad Solidaria y con Restitución de Derechos a las Personas en Situación de Calle

Ciudad Digital para la Equidad la Eficiencia y Participación Ciudadana

El documento es el siguiente:
""" + texto_ppg
)


resumen_ppg_2=response.text

In [65]:
#Pegar los resúmenes
resumen_ppg = limpiar_texto(resumen_ppg_1 + "\n\n" + resumen_ppg_2)

In [66]:
resumen_ppg_clean= limpiar_texto(resumen_ppg)

In [67]:
pdf_fd = FPDF()
pdf_fd.add_page()
pdf_fd.set_font("Arial", size = 12)
pdf_fd.multi_cell(0, 10, resumen_ppg_clean)
pdf_fd.output("resumen_ppg.pdf")

''